In [2]:
import os
import sys
WORKING_DIR = os.path.dirname(os.path.abspath('./'))
print(WORKING_DIR)
sys.path.append(WORKING_DIR)

/Users/lorenz/Repos/extractify/backend


In [3]:
import pandas as pd

df = pd.read_csv("../dataset/mantis_json_dataset_with_intents/mantis_utterance_category.csv")

In [5]:
import json

from entity_linking import EntityLinking
from foundation_models.chat_openai import AIModelType
from schema import topic_schema
current_model = AIModelType.MISTRAL_MIXTRAL_8x22B


async def link_entities(message: str):
    # generate function call schema
    schema = topic_schema
    
    # gemerate function calling schema
    llm_module = EntityLinking(schema=schema,model=current_model)
    
    print("current_model:", current_model)
    
    if current_model == AIModelType.MISTRAL_LARGE or current_model == AIModelType.MISTRAL_MIXTRAL_8x22B or current_model == AIModelType.MISTRAL_SMALL:
        filter_generator_output = llm_module.generate_sync(conversation=message)
        return filter_generator_output
    if current_model == AIModelType.GPT4_O or current_model == AIModelType.GPT4_TURBO or current_model == AIModelType.GPT4_O_MINI or current_model == AIModelType.GPT3 or current_model == AIModelType.GOOGLE_GEMINI_PRO or current_model == AIModelType.CLAUDE_OPUS or current_model == AIModelType.CLAUDE_SONNET:
        filter_generator_output = await llm_module.generate_response_generic(conversation=message)
    else:
        filter_generator_output = await llm_module.generate_async(conversation=message)
        
    print("filter_generator_output:", filter_generator_output)
    
    return filter_generator_output
        
    
# test the function
message = "What is the difference when speaking about 'LOB' and 'binary' data? Is it the same, in terms of binary data being stored in a separate filegroup, or are there differences?"
response = await link_entities(message)
print("response:", response)

# save to json file
with open('./temp.json', 'w') as f:
    json.dump(response, f, indent=4, ensure_ascii=False)

Using Mistral
current_model: AIModelType.MISTRAL_MIXTRAL_8x22B
Generate sync
promp 
Gegeben ist folgende Anfrage:s

Query:
What is the difference when speaking about 'LOB' and 'binary' data? Is it the same, in terms of binary data being stored in a separate filegroup, or are there differences?

Extrahiere die gewünschten Filterwerte aus der Konversation, nutze dabei nur die zur Verfügung stehenden Filterwerte. Bedenke aber auch implizite Filterwerte, die nicht explizit genannt werden,
aber aus dem Kontext hervorgehen und helfen dem Kunden das passende Produkt zu finden.

Denke dabei Schritt für Schritt vor und nutze die Funktionen, die dir zur Verfügung stehen. Erfinde keine neuen Werte in den Filter Werten.
Wenn du für das Gespräch keine passendes Filter Werte findest, gib ein leeres Array zurück.

response: {'category': 'dba'}


In [ ]:
# iterate over rows with iterrows()
import asyncio
import json

tasks = []

for index, row in df.iterrows():
    predicted_entities = row[4]

    if predicted_entities is not None and type(predicted_entities) == str:
        print("Already predicted entities")
        continue

    query = row[1]
    actual_category = row[2]
    print(actual_category)
    print(query)

    async def recognize_and_return_index(
        index, message: str
    ):
        (filter_generator_output, recognized_filters) = (
            await link_entities(
                message=message,
            )
        )
        
        print(recognized_filters)

        df.at[index, "Prediction"] = json.dumps(recognized_filters)
        return index

    print(index)
    
    try:
        await recognize_and_return_index(
        index=index,
        message=query,
    )
    except Exception as e:
        print(e)
        continue

    # wait to avoid rate limiting
    await asyncio.sleep(5)

 
    # write to csv
    df.to_json(f"prediction_{current_model}_v1.json", index=False, orient="records")


print(len(tasks))

In [ ]:
def normalize_string(s):
    if type(s) == int:
        return str(s)
    """Normalize a string by removing spaces, converting to lowercase, and replacing dashes."""
    return s.replace(' ', '').replace('-', '').replace('/','').lower()

mapping = {
    "ultrahd4k": "ultrahd",
    "größerals84zoll": 'gr\\u00f6\\u00dferals84zoll',
    "größerals19zoll": 'gr\\u00f6\\u00dferals19zoll',
    "144hz": "144hz",
}

speicherkapazitaet_mapping = {
    "256" : "256gb",
    "512" : "512gb",
    "1024" : "1024gb",
    "128" : "128gb",
    "64" : "64gb",
}

bildwiederholungsfrequenz_mapping = {
    "60" : "60hz",
    "120" : "120hz",
    "144" : "144hz",
    "165": "165hz",
    "240" : "240hz",
}


def split_entities_in_separated_entities(filter) -> list[tuple]:
    separated_entities = []
    for (key,values) in filter.items():
        for value in values:
            normalized_value = normalize_string(value)
            mapped_value = mapping.get(normalized_value, normalized_value)
            if key == "speicherkapazitaet":
                mapped_value = speicherkapazitaet_mapping.get(normalized_value, normalized_value)
            if key == "bildwiederholungsfrequenz":
                mapped_value = bildwiederholungsfrequenz_mapping.get(normalized_value, normalized_value)
            if key == "preis":
                mapped_value = value
                print("preis", value)
            separated_entities.append((key, mapped_value))
            
    return separated_entities

def calculate_tp_fn_fp(actual: list[tuple], predicted: list[tuple]):
    """
    Calculates the true positives (tp), false negatives (fn), and false positives (fp) 
    for a given set of actual and predicted entities.

    Args:
        actual (list[tuple]): The list of actual entities.
        predicted (list[tuple]): The list of predicted entities.

    Returns:
        tuple: A tuple containing the counts of tp, fn, fp, and tn.
    """

    tp = 0
    fp = 0
    fn = 0
    tn = 0

    fp_entities = []
    fn_entities = []

    if len(actual) == 0 and len(predicted) == 0:
        tn = 1
        return tp, fn, fp, tn, fp_entities,fn_entities


    for entity in actual:
        if entity in predicted:
            tp += 1
        else:
            fn += 1
            fn_entities.append((actual, entity))

    for entity in predicted:
        if entity not in actual:
            fp += 1
            fp_entities.append((actual, entity))

    return tp, fn, fp, tn, fp_entities, fn_entities


def calculate_precision_recall_f1(tp, fn, fp):
    if tp + fp > 0:
        precision = tp / (tp + fp)
    else:
        precision = 0

    if tp + fn > 0:
        recall = tp / (tp + fn)
    else:
        recall = 0

    if precision + recall > 0:
        f1 = 2 * (precision * recall) / (precision + recall)
    else:
        f1 = 0

    return precision, recall, f1

In [ ]:
# import pandas as pd
import json
df = pd.read_json(f"prediction_LLAMA_3_70B_v1.json")

In [ ]:

import math
import numpy as np

total_tp = 0
total_fn = 0
total_fp = 0
total_tn = 0

false_positives = []
false_negatives = []

for index, row in df.iterrows():
    query = row[1]
    print(query)
    print(row[3])
    actual_entities = json.loads(row[3])
    actual_category = row[2]

    if actual_category == "Other":
        continue

    predicted_entities = row[4]

    # check if predicted entities is nan
    if isinstance(predicted_entities, float) and math.isnan(predicted_entities):
        predicted_entities = []
    elif type(predicted_entities) == dict:
        continue
    elif predicted_entities is None:
        predicted_entities = []
    else:
        predicted_entities = json.loads(predicted_entities)

    # convert predicted entities to dict
    converted_predicted_entities = {}
    
    for entity in predicted_entities:
        if 'values' in entity:
            converted_predicted_entities.update({entity['id'].lower(): entity['values']})
        else:
            if 'maximum' in entity or 'minimum' in entity:
                converted_predicted_entities.update({entity['id'].lower(): [str(entity.get("minimum",0)),str(entity.get("maximum",0))]})


    # print("converted_predicted_entities",converted_predicted_entities)
    # process predicted entities
    predicted_entities = split_entities_in_separated_entities(converted_predicted_entities)
    actual_entities = split_entities_in_separated_entities(actual_entities)
    
    print("predicted_entities",predicted_entities)
    print("actual_entities",actual_entities)
    
    # add category tuple to actual entities
    actual_entities.append(("kategorie", normalize_string(actual_category)))
    
    
    (tp, fn, fp, tn, fp_entities,fn_entities) = calculate_tp_fn_fp(actual_entities, predicted_entities)
    total_tp += tp
    total_fn += fn
    total_fp += fp
    total_tn += tn

    if len(fp_entities) > 0:
        false_positives.append((query, fp_entities))

    if len(fn_entities) > 0:
        false_negatives.append((query, fn_entities))


# calculate precision, recall, and f1
precision, recall, f1 = calculate_precision_recall_f1(total_tp, total_fn, total_fp)
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1: {f1}")
print("total tn", total_tn)

    

In [ ]:
# write false positives to file
with open('false_positives.txt', 'w') as f:
    for fp in false_positives:
        f.write(f"{fp}\n")

In [ ]:
# write false negatives to file
with open('false_negatives.txt', 'w') as f:
    for fn in false_negatives:
        f.write(f"{fn}\n")

In [ ]:
# visualize the results in a confusion matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# create confusion matrix
conf_matrix = np.array([[total_tp, total_fn], [total_fp, total_tn]])

# create heatmap
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['True', 'False'], yticklabels=['True', 'False'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()


We have lots of false negatives, as the model does not have all filter ids in schema for Function Call Model. We use custom filter ids for the experiment, to give them a fair chance.

In [ ]:
def process_entities(x):
    if isinstance(x, float) and math.isnan(x):
        x = []
    else:
        x = json.loads(x)

    return {entity["id"]: entity["values"] for entity in x}


# Detect all filters that are never recognized
actual_entities = df["Entities"].apply(json.loads).explode().dropna().unique()

predicted_entities = df["Recognized_Filters_Prediction"].apply(process_entities)
predicted_entities = predicted_entities.explode().dropna().unique()

never_recognized = [entity for entity in actual_entities if entity not in predicted_entities]
print(never_recognized)

In [ ]:
grouped = df.groupby("Category")
unique_entities = grouped['Entities'].apply(lambda x: x.apply(json.loads).explode().dropna().unique())
    
# store to json
unique_entities.to_json('filters_per_category.json')